In [1]:
from reservoir_rls import *
from lorenz63 import *
from sklearn import preprocessing
from scipy.stats import sem
import cma
from scipy.stats import kstest, norm, shapiro
from multiprocessing import Pool
from min_func import *
from tqdm import tqdm


# from jupyterthemes import jtplot
# jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

In [2]:
data_length = 200000
step = 0.05
r_t = lambda x: r_t_const(x)
dxdt = lambda x,t,r_t: dxdt_lorenz(x,t,r_t)
data = getLorenzData(data_length, r_t, dxdt, sample_tau = step)

sync_length = 200
train_length = 800
pred_length = 500

# scaler = preprocessing.StandardScaler().fit(data)
# scaled_data = scaler.transform(data)
scaled_data = np.copy(data)
scaled_data = np.ascontiguousarray(scaled_data)

-0.05210027118992914
18.92542301780254
-19.022413088306337


In [3]:
num_nodes = 210
num_tests = 100
res_seed = 1
base_res = reservoir(3,num_nodes,input_weight = 1, spectral_radius = 1, seed = res_seed)
mask = ['input_weight', 'regularization']
x0 = np.array([5, 5])
min_func = lambda x: vt_min_function_norm(scaled_data, x, mask, base_res.Win, base_res.A, \
     num_tests = num_tests,  num_nodes = num_nodes, pred_length = pred_length, train_length = train_length)
sigma = 2

In [4]:
print(cma.CMAOptions())

{'AdaptSigma': 'True  # or False or any CMAAdaptSigmaBase class e.g. CMAAdaptSigmaTPA, CMAAdaptSigmaCSA', 'CMA_active': 'True  # negative update, conducted after the original update', 'CMA_cmean': '1  # learning rate for the mean value', 'CMA_const_trace': 'False  # normalize trace, 1, True, "arithm", "geom", "aeig", "geig" are valid', 'CMA_diagonal': '0*100*N/popsize**0.5  # nb of iterations with diagonal covariance matrix, True for always', 'CMA_eigenmethod': 'np.linalg.eigh  # or cma.utilities.math.eig or pygsl.eigen.eigenvectors', 'CMA_elitist': 'False  #v or "initial" or True, elitism likely impairs global search performance', 'CMA_injections_threshold_keep_len': '1  #v keep length if Mahalanobis length is below the given relative threshold', 'CMA_mirrors': 'popsize < 6  # values <0.5 are interpreted as fraction, values >1 as numbers (rounded), otherwise about 0.16 is used', 'CMA_mirrormethod': '2  # 0=unconditional, 1=selective, 2=selective with delay', 'CMA_mu': 'None  # parents

In [6]:
opts = cma.CMAOptions()
opts.set('popsize',4*x0.size)
opts.set('bounds', [0,10])
opts.set('verb_disp', 1)
opts.set('seed', 5)
opts.set('verb_filenameprefix','cmaes_norm_res%d\\' % res_seed)
# constraints = lambda x: [x[1] - 10, -x[1]]
results = cma.fmin(min_func, x0, sigma,options = opts)

(4_w,8)-aCMA-ES (mu_w=2.6,w_1=52%) in dimension 2 (seed=5, Fri Nov 27 13:26:10 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      8 -1.395000000000000e+02 1.0e+00 1.91e+00  1e+00  2e+00 0:28.4
    2     16 -1.740000000000000e+02 1.6e+00 2.04e+00  1e+00  3e+00 0:57.9
    3     24 -1.765000000000000e+02 2.4e+00 2.30e+00  9e-01  3e+00 1:27.1
    4     32 -1.345000000000000e+02 3.7e+00 2.49e+00  7e-01  4e+00 1:56.6
    5     40 -1.555000000000000e+02 5.7e+00 2.84e+00  6e-01  5e+00 2:26.9
    6     48 -1.390000000000000e+02 9.1e+00 2.69e+00  5e-01  5e+00 2:56.4
    7     56 -1.285000000000000e+02 1.1e+01 2.25e+00  3e-01  4e+00 3:26.3
    8     64 -1.330000000000000e+02 1.4e+01 1.82e+00  2e-01  3e+00 3:56.8
    9     72 -1.420000000000000e+02 1.5e+01 1.49e+00  2e-01  2e+00 4:27.4
   10     80 -1.665000000000000e+02 1.5e+01 1.19e+00  1e-01  1e+00 4:58.3
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
   11     88 -1.575000000000000

In [ ]:
test_reg = 8.5
weights = np.linspace(4,6,20)
forgets = np.linspace(1,4,20)
func_vals = np.zeros((weights.size, forgets.size))

with tqdm(total = weights.size * forgets.size) as pbar:
    for i in range(weights.size):
        for j in range(forgets.size):
            func_vals[i,j] = min_func(np.array([weights[i], test_reg, forgets[j]]))
            np.savetxt('rls_func_vals_reg%e.csv'%test_reg, func_vals, delimiter = ',')
            pbar.update(1)

In [ ]:
res2 = reservoir(3,210,forget = 1, input_weight = 0.25*0.05735187596920699, regularization = 10**(-9.310114193664688), seed = 3)
num_tests = 5000
pred_length = 500
train_length = 400
sync_length = 200
valid_times_norm = cross_validation_performance_resync(scaled_data,res2,num_tests,sync_length, \
   train_length, pred_length,train_method = 'Normal', progress = True)
print(np.median(valid_times_norm))

In [ ]:
valid_times_RLS = cross_validation_performance(scaled_data,res1,num_tests,sync_length, \
   train_length, pred_length,train_method = 'RLS')
print(np.median(valid_times_RLS))
valid_times_norm = cross_validation_performance(scaled_data,res2,num_tests,sync_length, \
   train_length, pred_length,train_method = 'Normal')
print(np.median(valid_times_norm))

In [ ]:
res1 = reservoir(3,300,forget = 0.985, input_weight = 0.01, regularization = 1e-8, \
     t_regularization = 5e-5, delta = 1e6, t_weighted = True)

In [ ]:
# valid_times_norm_array = np.loadtxt('Lorenz63Data/valid_times_norm_tweight_forget%f_reg%e_treg%e.csv' \
#     %(res1.forget, res1.regularization, res1.t_regularization), delimiter = ',')

In [ ]:
step = 0.05
base_data_length = 25000
data_length = int(base_data_length * step/(0.01))
r_t = lambda x: r_t_cosine(x)
dxdt = lambda x,t,r_t: dxdt_lorenz(x,t,r_t)
valid_times_norm_array = np.array([])

sync_length = 1000
num_tests = 300
train_length = 800
pred_length = 200
completed_seeds = valid_times_norm_array.shape[0]

# scaler = preprocessing.StandardScaler().fit(data)
# scaled_data = scaler.transform(data)
from tqdm import tqdm
with tqdm(total = num_tests) as pbar:
    for seed in range(num_tests):
        data = getLorenzData(data_length, r_t, dxdt, sample_tau = step, seed = seed)
        scaled_data = np.ascontiguousarray(data)
        valid_times_norm = cross_validation_performance_versust(scaled_data,res1,sync_length, \
               train_length, pred_length,pred_gap_length = 100, train_method = 'Normal', progress = False)
        if valid_times_norm_array.size == 0:
            valid_times_norm_array = np.copy(valid_times_norm)
        else:
            valid_times_norm_array = np.vstack((valid_times_norm_array,valid_times_norm))
        np.savetxt('Lorenz63Data/valid_times_norm_tweight_forget%f_reg%e_treg%e.csv' %(res1.forget, res1.regularization, res1.t_regularization), \
            valid_times_norm_array, delimiter = ',')
        pbar.update(1)

In [ ]:
np.savetxt('Lorenz63Data/valid_times_norm_tweight_forget%f_reg%e_treg%e.csv' %(res1.forget, res1.regularization, res1.t_regularization), \
    valid_times_norm_array, delimiter = ',')

In [ ]:
plt.plot(np.mean(valid_times_norm_array*0.05,axis = 0))
plt.ylim(0,5)
plt.show()

In [ ]:
res1 = reservoir(3,300,forget = 0.985, input_weight = 0.01, LM_regularization = 1e-10, \
     t_regularization = 0, delta = 1e6, t_weighted = False, RLS_reg_type = 'LM')
step = 0.05
base_data_length = 5000
data_length = int(base_data_length * step/(0.01))
r_t = lambda x: r_t_cosine(x)
dxdt = lambda x,t,r_t: dxdt_lorenz(x,t,r_t)
valid_times_norm_array = np.array([])

sync_length = 500
num_tests = 100
train_length = 2000
pred_length = 400

In [ ]:
data = getLorenzData(data_length, r_t, dxdt, sample_tau = step, seed = 1)
new_data = np.ascontiguousarray(data)

In [ ]:
res1.trainWoutRLS(scaled_data[2800:3800],200)
res1.valid_time(scaled_data[3800:4200],plot = True, error_bound = 3.2)

In [ ]:
valid_times_norm = cross_validation_performance_separated(scaled_data,res1,sync_length, \
               train_length, pred_length, train_method = 'RLS', progress = False)
print(np.median(valid_times_norm))

In [ ]:
# scaler = preprocessing.StandardScaler().fit(data)
# scaled_data = scaler.transform(data)
from tqdm import tqdm
with tqdm(total = num_tests) as pbar:
    for seed in range(num_tests):
        data = getLorenzData(data_length, r_t, dxdt, sample_tau = step, seed = seed)
        scaled_data = np.ascontiguousarray(data)
        valid_times_norm = cross_validation_performance_versust(scaled_data,res1,sync_length, \
               train_length, pred_length,pred_gap_length = 100, train_method = 'RLS', progress = False)
        if valid_times_norm_array.size == 0:
            valid_times_norm_array = np.copy(valid_times_norm)
        else:
            valid_times_norm_array = np.vstack((valid_times_norm_array,valid_times_norm))
        np.savetxt('Lorenz63Data/valid_times_rls_tweight_forget%f_reg%e_treg%e.csv' %(res1.forget, \
            res1.regularization, res1.t_regularization), valid_times_norm_array, delimiter = ',')
        pbar.update(1)

In [ ]:
plt.plot(np.mean(valid_times_norm_array*0.05,axis = 0))
plt.ylim(0,5)
plt.show()

In [ ]:
x = np.array([[1,2],[3,4]])
np.mean(x,axis = 1)